In [ ]:
from typing import List

import missingno as msno
import pandas as pd
from IPython.display import HTML
import numpy as np

from sklearn import datasets

from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score


import matplotlib.pyplot as plt

import seaborn as sns

sns.set_theme(style="whitegrid")
pd.set_option("display.max_rows", None)


In [ ]:
foods = pd.read_csv(
    "files/fr.openfoodfacts.org.products.csv", sep="\t", low_memory=False
)


## Dataset overview

In [ ]:
foods.shape


In [ ]:
foods.dtypes


In [ ]:
(foods.isnull().mean().round(2)).sort_values()


In [ ]:
%matplotlib inline
msno.bar(foods)
plt.axvline(0.5, color="r")


## Clean and filter features and product

In [ ]:
CATEGORICAL_FEATURES = [
    "url",
    "code",
    "creator",
    "created_datetime",
    "last_modified_datetime",
    "product_name",
    "generic_name",
    "packaging",
    "packaging_tags",
    "brands_tags",
    "brands",
    "categories",
    "categories_tags",
    "categories_fr",
    "origins",
    "origins_tags",
    "manufacturing_places",
    "manufacturing_places_tags",
    "labels",
    "labels_tags",
    "labels_fr",
    "emb_codes",
    "emb_codes_tags",
    "first_packaging_code_geo",
    "cities",
    "cities_tags",
    "purchase_places",
    "stores",
    "countries",
    "countries_tags",
    "countries_fr",
    "ingredients_text",
    "traces",
    "traces_tags",
    "no_nutriments",
    "additives",
    "additives_tags",
    "ingredients_from_palm_oil",
    "ingredients_from_palm_oil_tags",
    "ingredients_that_may_be_from_palm_oil",
    "ingredients_that_may_be_from_palm_oil_tags",
    "nutrition_grade_fr",
    "nutrition_grade_uk",
    "main_category",
    "main_category_fr",
    "image_url",
    "image_small_url",
    "allergens",
    "allergens_fr",
    "traces_fr",
    "additives_fr",
    "pnns_groups_1",
    "pnns_groups_2",
    "states",
    "states_tags",
    "states_fr",
]


In [ ]:
def clean_and_filter_features_and_product(
    df: pd.DataFrame, feature: str, sub_features=List[str]
):
    df[feature] = df[feature].replace(
        {
            "cereals": "Cereals",
            "fruits": "Fruits",
            "legumes": "Legumes",
            "pastries": "Pastries",
            "nuts": "Nuts",
            "vegetables": "Vegetables",
        }
    )

    categorical_foods = foods[CATEGORICAL_FEATURES]

    categorical_foods_mean_null_by_column = categorical_foods.isnull().mean()
    missing_percentage = categorical_foods_mean_null_by_column[feature] * 100

    if missing_percentage < 50:
        return display(
            HTML(
                f"""
                    <p style='color:orange;'>The feature {feature} has less than 50% of missing values ({round(missing_percentage, 1)} %).</p>
                    "<p style='color:red;'>Consider choosing another feature.</p>"
                     <hr style="border:1px solid #000;" />
                """
            )
        )
    display(
        HTML(
            f"""
                <p style='color:green;'>The feature {feature} has more than 50% of missing values ({round(missing_percentage, 1)} %).</p>
                <hr style="border:1px solid #000;" />
            """
        )
    )
    df1 = df.copy()
    %matplotlib inline
    msno.matrix(df1)
    plt.axvline(0.5, color="r")

    df = df.dropna(subset=[feature])
    df = df[df[feature] != "unknown"]

    sub_features_foods = df[sub_features]

    sub_features_foods_mean_null_by_column = sub_features_foods.isnull().mean()

    at_least_sub_feature_with_missing_values_gt_50_percent = False
    for sub_feature in sub_features:
        missing_percentage = sub_features_foods_mean_null_by_column[sub_feature] * 100

        if missing_percentage > 50:
            at_least_sub_feature_with_missing_values_gt_50_percent = True
            display(
                HTML(
                    f"<p style='color:orange;'>The feature {sub_feature} has more than 50% of missing values ({round(missing_percentage, 1)} %).</p>"
                )
            )
        else:
            display(
                HTML(
                    f"<p style='color:green;'>The feature {sub_feature} has less than 50% of missing values ({round(missing_percentage, 1)} %).</p>"
                )
            )

    if at_least_sub_feature_with_missing_values_gt_50_percent:
        display(
            HTML(
                "<p style='color:red;'>At least one feature has more than 50% of missing values. Consider choosing another feature.</p>"
            )
        )

    sub_features_foods = sub_features_foods.copy()

    sub_features_foods = sub_features_foods.dropna(
        subset=[
            "product_name",
            "brands",
            "packaging",
            "quantity",
            "countries",
            "ingredients_text",
        ]
    )

    sub_features_foods = sub_features_foods.dropna(
        subset=[
            "proteins_100g",
            "carbohydrates_100g",
            "sugars_100g",
            "fat_100g",
            "saturated-fat_100g",
        ],
        how="all",
    )

    sub_features_foods = sub_features_foods.drop_duplicates(
        subset=[
            "product_name",
            "brands",
            "packaging",
            "quantity",
            "countries",
            "ingredients_text",
            "proteins_100g",
            "carbohydrates_100g",
            "sugars_100g",
            "fat_100g",
            "saturated-fat_100g",
        ],
        keep=False,
    )
    display(msno.bar(sub_features_foods))
    return sub_features_foods, df1


cleaned_df, df1 = clean_and_filter_features_and_product(
    foods,
    "pnns_groups_2",
    [
        "product_name",
        "pnns_groups_2",
        "nutrition_grade_fr",
        "brands",
        "packaging",
        "quantity",
        "countries",
        "ingredients_text",
        "code",
        "proteins_100g",
        "carbohydrates_100g",
        "sugars_100g",
        "fat_100g",
        "saturated-fat_100g",
    ],
)


Je ne garde pas les colonnes salt_100g et energy_100g.

La colonne salt_100g contient des valeurs qui correspondent de temps en temps au pourcentage de sodium ou de sel en mg. Je n'ai pas trouvé de relation qui puisse m'aider à automiser la conversion.

La colonne energy_100g contient des valeurs soit en kJ soit en kcal. Je n'ai aussi pas trouvé de moyen pour automatiser la conversion.

## Manage outliers


In [ ]:
columns_to_check_outliers = [
    "proteins_100g",
    "carbohydrates_100g",
    "sugars_100g",
    "fat_100g",
    "saturated-fat_100g",
]

cleaned_df[columns_to_check_outliers] = cleaned_df[columns_to_check_outliers].where(
    (cleaned_df[columns_to_check_outliers] >= 0)
    & (cleaned_df[columns_to_check_outliers] <= 100),
    np.nan,
)


food_groups = cleaned_df.pnns_groups_2.unique()

for column in columns_to_check_outliers:
    outlier_column = f"{column}_is_outlier"
    cleaned_df[outlier_column] = False

    grouped = cleaned_df.groupby("pnns_groups_2")

    for name, group in grouped:
        q1 = group[column].quantile(0.25)
        q3 = group[column].quantile(0.75)
        iqr = q3 - q1

        lower_bounce = q1 - 1.5 * iqr
        upper_bounce = q3 + 1.5 * iqr

        outliers = (group[column] < lower_bounce) | (group[column] > upper_bounce)

        cleaned_df.loc[group.index, outlier_column] = outliers


## Manage empty values


In [ ]:
for group in food_groups:
    for column in columns_to_check_outliers:
        skewness = cleaned_df[cleaned_df["pnns_groups_2"] == group][column].skew()
        if np.isnan(skewness):
            continue
        elif abs(skewness) >= 1:
            strategy = "median"
        elif abs(skewness) < 0.5:
            strategy = "mean"
        else:
            strategy = "median"

        imputer = SimpleImputer(strategy=strategy)
        group_data = cleaned_df[cleaned_df["pnns_groups_2"] == group]
        imputed_values = imputer.fit_transform(group_data[[column]])
        cleaned_df.loc[cleaned_df["pnns_groups_2"] == group, column] = imputed_values

        imputer = SimpleImputer(strategy="most_frequent")
        group_data = cleaned_df[cleaned_df["pnns_groups_2"] == group]
        imputed_values = imputer.fit_transform(group_data[["nutrition_grade_fr"]])
        cleaned_df.loc[cleaned_df["pnns_groups_2"] == group, "nutrition_grade_fr"] = (
            imputed_values
        )


## Univariate and bivariate analysis


### Univariate analysis


In [ ]:
top_5_groups_count = cleaned_df["pnns_groups_2"].value_counts().head(5)
display(top_5_groups_count)
analysis_df = cleaned_df.copy()

columns_to_analyse = (
    "proteins_100g",
    "carbohydrates_100g",
    "sugars_100g",
    "fat_100g",
    "saturated-fat_100g",
)


def plot_histogram(df):
    for column in columns_to_analyse:
        plt.figure(figsize=(15, 6))
        plt.hist(
            df[column],
        )
        plt.title(f"Histogram of {column}")
        plt.xlabel(column)
        plt.ylabel("Frequency")
        plt.grid(True)
        plt.show()


def plot_box(df):
    for column in columns_to_analyse:
        plt.figure(figsize=(15, 6))
        plt.boxplot(df[column], vert=False)
        plt.title(f"Boxplot of {column}")
        plt.xlabel(column)
        plt.ylabel("Frequency")
        plt.grid(True)
        plt.show()


In [ ]:
plot_histogram(cleaned_df)


In [ ]:
plot_box(cleaned_df)


In [ ]:
def plot_pie_chart(df):
    # Calculate the counts of each category
    category_counts = df["nutrition_grade_fr"].value_counts()

    # Plot the pie chart
    plt.figure(figsize=(8, 6))
    plt.pie(
        category_counts,
        labels=category_counts.index,
        autopct="%1.1f%%",
        startangle=90,
        colors=plt.cm.Paired.colors,
    )
    plt.title("Pie Chart of nutrition grade")
    plt.axis("equal")  # Equal aspect ratio ensures that pie is drawn as a circle.
    plt.show()


# Example usage: Plot pie chart for a categorical column
plot_pie_chart(cleaned_df)


## Bi variate

In [ ]:
# cleaned_df = cleaned_df[cleaned_df["pnns_groups_2"] == "One-dish meals"][[ "proteins_100g", "carbohydrates_100g", "sugars_100g", "fat_100g"]]
# tmp_df = cleaned_df[[ "proteins_100g", "carbohydrates_100g", "sugars_100g", "fat_100g"]]

nutrition_grade_encoder = OrdinalEncoder(categories=[["a", "b", "c", "d", "e"]])

cleaned_df["nutrition_grade_fr_numeric"] = nutrition_grade_encoder.fit_transform(
    cleaned_df[["nutrition_grade_fr"]]
)
corr_matrix = cleaned_df[
    [
        "proteins_100g",
        "carbohydrates_100g",
        "sugars_100g",
        "fat_100g",
        "saturated-fat_100g",
        "nutrition_grade_fr_numeric",
    ]
].corr()

plt.figure(figsize=(10, 6))
sns.heatmap(corr_matrix, annot=True)
plt.title("Heatmap of Nutritional Values by Product")
plt.show()


In [ ]:
def scatter_food_group(df):
    for column in columns_to_analyse:
        plt.figure(figsize=(15, 7))
        plt.ylabel("Food groups")
        plt.xlabel(column.split("_")[0])

        sns.violinplot(y="pnns_groups_2", x=column, data=cleaned_df)


scatter_food_group(cleaned_df)


## Multi variate

### ACP

In [ ]:
scaled_df = cleaned_df[
    [
        "proteins_100g",
        "carbohydrates_100g",
        "sugars_100g",
        "fat_100g",
        "saturated-fat_100g",
    ]
]


scaler = StandardScaler()
data_scaled = scaler.fit_transform(scaled_df)

idx = ["mean", "std"]
display(pd.DataFrame(data_scaled).describe().round(2).loc[idx, :])

n_components = 5
pca = PCA(n_components=n_components)
pca_components = pca.fit(data_scaled)

# Créer un DataFrame avec les deux composantes principales
# pca_df = pd.DataFrame(data=pca_components, columns=['PC1', 'PC2'])

display(pca_components)


In [ ]:
pca.explained_variance_ratio_


In [ ]:
scree = (pca.explained_variance_ratio_ * 100).round(2)
scree


In [ ]:
scree_cum = scree.cumsum().round()
scree_cum


In [ ]:
x_list = range(1, n_components + 1)
list(x_list)


In [ ]:
plt.bar(x_list, scree)
plt.plot(x_list, scree_cum, c="red", marker="o")
plt.xlabel("rang de l'axe d'inertie")
plt.ylabel("pourcentage d'inertie")
plt.title("Eboulis des valeurs propres")
plt.show(block=False)


In [ ]:
pcs = pca.components_


In [ ]:
pcs = pd.DataFrame(pcs)
pcs


In [ ]:
fig, ax = plt.subplots(figsize=(20, 6))
sns.heatmap(pcs.T, vmin=-1, vmax=1, annot=True, cmap="coolwarm", fmt="0.2f")


In [ ]:
x, y = 0, 1

fig, ax = plt.subplots(figsize=(10, 9))
for i in range(0, pca.components_.shape[1]):
    ax.arrow(
        0,
        0,  # Start the arrow at the origin
        pca.components_[0, i],  # 0 for PC1
        pca.components_[1, i],  # 1 for PC2
        head_width=0.07,
        head_length=0.07,
        width=0.02,
    )

    plt.text(pca.components_[0, i] + 0.05, pca.components_[1, i] + 0.05, features[i])

# affichage des lignes horizontales et verticales
plt.plot([-1, 1], [0, 0], color="grey", ls="--")
plt.plot([0, 0], [-1, 1], color="grey", ls="--")


# nom des axes, avec le pourcentage d'inertie expliqué
plt.xlabel("F{} ({}%)".format(x + 1, round(100 * pca.explained_variance_ratio_[x], 1)))
plt.ylabel("F{} ({}%)".format(y + 1, round(100 * pca.explained_variance_ratio_[y], 1)))

plt.title("Cercle des corrélations (F{} et F{})".format(x + 1, y + 1))


an = np.linspace(0, 2 * np.pi, 100)
plt.plot(np.cos(an), np.sin(an))  # Add a unit circle for scale
plt.axis("equal")
plt.show(block=False)


In [ ]:
import numpy as np  # Importing numpy for linspace and trigonometric functions

# Re-run the plot code after fixing the missing numpy import
x, y = 1, 2  # For F2 and F3

fig, ax = plt.subplots(figsize=(10, 9))

# Plotting arrows for F2 (x=1) and F3 (y=2)
for i in range(0, pca.components_.shape[1]):
    ax.arrow(
        0,
        0,  # Start the arrow at the origin
        pca.components_[x, i],  # F2 (PC2)
        pca.components_[y, i],  # F3 (PC3)
        head_width=0.07,
        head_length=0.07,
        width=0.02,
    )

    # Adding feature names next to the arrows
    plt.text(pca.components_[x, i] + 0.05, pca.components_[y, i] + 0.05, features[i])

# Displaying horizontal and vertical dashed lines at the origin
plt.plot([-1, 1], [0, 0], color="grey", ls="--")
plt.plot([0, 0], [-1, 1], color="grey", ls="--")

# Labeling the axes with the percentage of variance explained
plt.xlabel("F{} ({}%)".format(x + 1, round(100 * pca.explained_variance_ratio_[x], 1)))
plt.ylabel("F{} ({}%)".format(y + 1, round(100 * pca.explained_variance_ratio_[y], 1)))

plt.title("Cercle des corrélations (F{} et F{})".format(x + 1, y + 1))

# Add a unit circle for scale
an = np.linspace(0, 2 * np.pi, 100)
plt.plot(np.cos(an), np.sin(an))  # Circle with radius 1
plt.axis("equal")

# Show the plot
plt.show()


### K-means

In [ ]:
intertia = []


In [ ]:
k_list = range(1, 10)


In [ ]:
X = cleaned_df[
    [
        "proteins_100g",
        "carbohydrates_100g",
        "sugars_100g",
        "fat_100g",
        "saturated-fat_100g",
    ]
]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
for i in k_list:
    kmeans = KMeans(n_clusters=i)
    kmeans.fit(X)
    intertia.append(kmeans.inertia_)

intertia


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 6))

ax.set_ylabel("inertia")
ax.set_xlabel("n_cluster")
ax = plt.plot(k_list, intertia)


In [ ]:
silhouette_scores = []

# Parcours des valeurs de k (nombre de clusters)
range_n_clusters = range(2, 11)  # Testons pour k de 2 à 10

for n_clusters in range_n_clusters:
    # Créer un modèle K-means avec k clusters
    kmeans = KMeans(n_clusters=n_clusters)
    cluster_labels = kmeans.fit_predict(X)

    # Calculer le coefficient de silhouette pour cette valeur de k
    silhouette_avg = silhouette_score(X, cluster_labels)
    silhouette_scores.append(silhouette_avg)
    print(f"Pour k={n_clusters}, le score de silhouette moyen est {silhouette_avg}")

# Affichage du graphique des scores de silhouette
plt.plot(range_n_clusters, silhouette_scores, marker="o")
plt.title("Score de silhouette pour différents k")
plt.xlabel("Nombre de clusters k")
plt.ylabel("Score de silhouette moyen")
plt.show()


In [ ]:
n_clusters = 2
kmeans = KMeans(n_clusters=n_clusters)
kmeans.fit(X)


In [ ]:
kmeans.labels_


In [ ]:
dd = {i: j for i, j in enumerate(list("ab"))}
dd


In [ ]:
labels = [dd[i] for i in kmeans.labels_]
labels[:10]


In [ ]:
X["cluster"] = labels


In [ ]:
sns.pairplot(X, hue="cluster")


In [ ]:
# Sélection des variables
X = cleaned_df[
    [
        "proteins_100g",
        "carbohydrates_100g",
        "sugars_100g",
        "fat_100g",
        "saturated-fat_100g",
    ]
]

# Standardisation des données
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Application du K-means sur les données standardisées
kmeans = KMeans(n_clusters=2, random_state=42)
clusters = kmeans.fit_predict(X_scaled)

# Ajout des labels de clusters au DataFrame
X["Cluster"] = clusters

# Application de l'ACP pour la visualisation
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)

# Création d'un DataFrame avec les composantes principales et les clusters
pca_df = pd.DataFrame(data=X_pca, columns=["F1", "F2", "F3"])
pca_df["Cluster"] = clusters

# Visualisation
fig, ax = plt.subplots(figsize=(12, 6))

scatter = ax.scatter(
    pca_df["F1"], pca_df["F2"], c=pca_df["Cluster"], cmap="viridis", alpha=0.7
)

# Ajout des centroïdes projetés
centroids = kmeans.cluster_centers_
centroids_pca = pca.transform(centroids)
ax.scatter(
    centroids_pca[:, 0],
    centroids_pca[:, 1],
    c="red",
    s=100,
    marker="X",
    label="Centroïdes",
)

# Étiquettes et titre
ax.set_xlabel("Composante principale 1 (F1)")
ax.set_ylabel("Composante principale 2 (F2)")
ax.set_title("Clusters visualisés sur les composantes principales après K-means")

# Légende
legend_labels = ["Cluster 0", "Cluster 1"]
handles = [
    plt.Line2D(
        [],
        [],
        marker="o",
        color="w",
        label=label,
        markersize=10,
        markerfacecolor=scatter.cmap(scatter.norm(i)),
    )
    for i, label in enumerate(legend_labels)
]
handles.append(
    plt.Line2D(
        [],
        [],
        marker="X",
        color="w",
        label="Centroïdes",
        markersize=10,
        markerfacecolor="red",
    )
)
ax.legend(handles=handles, title="Légende")

plt.show()


### ANOVA

In [ ]:
X = "pnns_groups_2"  # qualitative

features = [
    "proteins_100g",
    "carbohydrates_100g",
    "sugars_100g",
    "fat_100g",
    "saturated-fat_100g",
]
for f in features:
    # On ne garde que les dépenses
    sous_echantillon = cleaned_df[
        cleaned_df[f] > 0
    ].copy()  # On remet les dépenses en positif
    # On n'étudie pas les loyers car trop gros:

    modalites = sous_echantillon[X].unique()
    groupes = []
    for m in modalites:
        groupes.append(sous_echantillon[sous_echantillon[X] == m][f])

    # Propriétés graphiques (pas très importantes)
    medianprops = {"color": "black"}
    meanprops = {
        "marker": "o",
        "markeredgecolor": "black",
        "markerfacecolor": "firebrick",
    }

    plt.boxplot(
        groupes,
        labels=modalites,
        showfliers=False,
        medianprops=medianprops,
        vert=False,
        patch_artist=True,
        showmeans=True,
        meanprops=meanprops,
    )
    plt.title(f"Boxplot of {f}")
    plt.show()


In [ ]:
sous_echantillon = cleaned_df


def eta_squared(x, y):
    moyenne_y = y.mean()
    classes = []
    for classe in x.unique():
        yi_classe = y[x == classe]
        classes.append({"ni": len(yi_classe), "moyenne_classe": yi_classe.mean()})
    SCT = sum([(yj - moyenne_y) ** 2 for yj in y])
    SCE = sum([c["ni"] * (c["moyenne_classe"] - moyenne_y) ** 2 for c in classes])
    return SCE / SCT


In [ ]:
features = [
    "proteins_100g",
    "carbohydrates_100g",
    "sugars_100g",
    "fat_100g",
    "saturated-fat_100g",
]

for f in features:
    sous_echantillon = cleaned_df[
        cleaned_df[f] > 0
    ].copy()  # On remet les dépenses en positif
    print(
        f"Eta squared for {f} is {eta_squared(sous_echantillon[X],sous_echantillon[f])}"
    )


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# Fonction principale pour lancer l'EDA
def eda(data):
    # 1. Aperçu des données
    print("Aperçu des données :")
    print(data.head(), "\n")
    print("Informations sur les colonnes et types :")
    print(data.info(), "\n")
    print("Résumé statistique des données numériques :")
    print(data.describe(), "\n")
    print("Résumé des données catégorielles :")
    print(data.describe(include="object"), "\n")

    # 2. Valeurs manquantes
    print("Valeurs manquantes :")
    missing_data = data.isnull().sum()
    print(missing_data[missing_data > 0], "\n")

    # Visualisation des valeurs manquantes
    plt.figure(figsize=(10, 6))
    sns.heatmap(data.isnull(), cbar=False, cmap="viridis")
    plt.title("Valeurs manquantes dans le jeu de données")
    plt.show()

    # 3. Distribution des variables numériques
    numeric_cols = data.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        plt.figure(figsize=(10, 6))
        sns.histplot(data[col], kde=True)
        plt.title(f"Distribution de {col}")
        plt.show()

    # 4. Analyse des variables catégorielles
    categorical_cols = data.select_dtypes(include=["object"]).columns
    for col in categorical_cols:
        plt.figure(figsize=(10, 6))
        sns.countplot(y=data[col], order=data[col].value_counts().index)
        plt.title(f"Répartition de {col}")
        plt.show()

    # 5. Matrice de corrélation pour les variables numériques
    if len(numeric_cols) > 1:
        plt.figure(figsize=(12, 8))
        sns.heatmap(data[numeric_cols].corr(), annot=True, fmt=".2f", cmap="coolwarm")
        plt.title("Matrice de corrélation des variables numériques")
        plt.show()

    # 6. Outliers (valeurs aberrantes) avec Boxplot
    for col in numeric_cols:
        plt.figure(figsize=(10, 6))
        sns.boxplot(x=data[col])
        plt.title(f"Boxplot de {col}")
        plt.show()


# Utilisation : appelle cette fonction avec ton DataFrame
# Exemple : eda(df)

eda(cleaned_df)
